# 1. 학습을 위한 데이터 구분 및 디렉토리 생성

In [1]:
import os
import pandas as pd
import random
import time
import string
from sklearn.model_selection import train_test_split

from pyjarowinkler import distance

In [2]:
base_path = 'LearningData'

clms = ['N/A','low', 'num','low&num', 
        'up', 'up&low','up&num', 'up&num&low',
        'spc', 'spc&low', 'spc&num', 'spc&num&low',
        'spc&up', 'spc&up&low', 'spc&up&num', 'all']

In [3]:
type_1 = [clms[1]]           # low -> low
type_2 = [clms[1],clms[3]]   # low -> low&num
type_3 = [clms[1],[clms[5],clms[9]]]   # low -> low&up, low&spc
type_4 = [clms[1],[clms[7],clms[11],clms[13]]]   # low -> low&up&num, low&num&spc, low&up&spc
type_5 = [clms[2]]           # num -> num   
type_6 = [clms[2],clms[3]]   # num -> low&num

extra_path = '/letter_specific_data'
type_1_path = os.path.join(base_path, 'type_1'+extra_path)
type_2_path = os.path.join(base_path, 'type_2'+extra_path)
type_3_path = os.path.join(base_path, 'type_3'+extra_path)
type_4_path = os.path.join(base_path, 'type_4'+extra_path)
type_5_path = os.path.join(base_path, 'type_5'+extra_path)
type_6_path = os.path.join(base_path, 'type_6'+extra_path)

if not os.path.exists(type_1_path):os.makedirs(type_1_path)
if not os.path.exists(type_2_path):os.makedirs(type_2_path)
if not os.path.exists(type_3_path):os.makedirs(type_3_path)
if not os.path.exists(type_4_path):os.makedirs(type_4_path)    
if not os.path.exists(type_5_path):os.makedirs(type_5_path)
if not os.path.exists(type_6_path):os.makedirs(type_6_path)    

In [4]:
letter = list(string.ascii_lowercase) +['num']

In [5]:
def delete_single(src):
    
    if src <2:
        return np.nan
    else:
        return src
    
def SplitData(listin):
    
    length = len(listin)
    listout = list()
    
    tmp = random.sample(listin, length)
        
    for i in range(length-1):
        tmp_list = [tmp[0], tmp[i+1]]
        listout.append(tmp_list)
    
    return listout

def DropReindex(learn_data, TYPE):
    
    learn_data = learn_data.dropna(how='any').reset_index(drop=True)
    learn_data['length'] = learn_data[TYPE[0]].str.len()
    learn_data['length'] = learn_data['length'].apply(delete_single)

    learn_data = learn_data.dropna(how='any')
    learn_data = learn_data.drop('length', axis = 1)
    learn_data = learn_data.reset_index(drop=True)

    return learn_data

def ListExtend(listin):
    
    tmp = list()
    
    for i in range(len(listin)):
        out = listin['Splitted'][i]
        tmp.extend(out)
        
        if i%100000==0:
            print(i)

    return tmp

In [6]:
def DFandSave(learn_data, TYPE ,char):
    
    base = './LearningData/type_'+ str(TYPE)+ '/letter_specific_data/'
    save_path = base+str(char)+'.pickle'
    
    data_columns = ['src', 'tgt']
    pw_data = pd.DataFrame(learn_data, columns=data_columns)
    pw_data.to_pickle(save_path)
    
    print(char, 'Data Saved', 'length=',len(pw_data))

In [7]:
def SplitData_Multi(data_src, data_tgt):
    
    listout = list()
    
    for i in range(len(data_src)):
        src = data_src[i]
        
        for j in range(len(data_tgt)):
            tgt = data_tgt[j]
            tmp = [src,tgt]
            listout.append(tmp)
    
    return listout

# 2. 데이터 Split&Save

# Type1

In [ ]:
for i in range(0,27):
    char = letter[i]
    print(char, 'Data Begin')
    
    # 데이터 로드
    load_path = './Preprocessed/' +str(char)+'.pickle'
    pw_df = pd.read_pickle(load_path)
    
    # 원하는 타입 추출
    learn_data = pw_df[type_1]
    print('length of original learn data:',len(learn_data))
    
    # 불필요한 데이터 drop 및 reindex
    learn_data = DropReindex(learn_data)
    print('length of processed learn data:',len(learn_data))
    
    # 길이가 2 이상인 데이터 나누기
    learn_data['Splitted'] = learn_data['low'].apply(SplitData)
    print('Data Splitted')
    
    # 나뉘어진 데이터 이어 붙이기 
    learn_data= ListExtend(learn_data)
    
    # src, tgt 데이터 프래임 생성 및 파일 저장
    DFandSave(learn_data,1,char)
    print('')
    

In [ ]:
# 올바른 전처리 결과 확인용
pd.read_pickle('./LearningData/type_1/letter_specific_data/k.pickle')

In [ ]:
# 전처리된 데이터 a~z, num 들을 로드한 후 1개의 df로 결합

data_columns = ['src', 'tgt']
pw_data = pd.DataFrame(columns=data_columns)

for i in range(0,27):
    char = letter[i]
    print(char, 'Data load and Append')
    
    load_path = './LearningData/type_1/letter_specific_data/'+str(char)+'.pickle'
    temp = pd.read_pickle(load_path)
    
    print(len(temp))
    pw_data = pw_data.append(temp)    

In [ ]:
# 데이터 저장

In [ ]:
## type 1
save_path = type_1_path

# 행의 값중 하나라도 nan인 경우 그 행을 없앤다., reindex
#pw_data = pw_data.dropna(how='any').reset_index(drop=True)

print('%s 갯수 : %d\n' % (type_1, len(pw_data)))
print(pw_data[:5])

file_name_src = pw_data.columns[0]
file_name_tgt = pw_data.columns[1]

print('%s -> %s'%(file_name_src, file_name_tgt))

# 하나는 txt, 하나는 pickle로 저장한다.

# src, tgt 모두 저장(txt)
path = os.path.join(save_path, 'pw_all.txt')
pw_data.to_csv(path, header=False, index=False )
# src, tgt 모두 저장(csv)
path1 = os.path.join(save_path, 'pw_all.pickle')
pw_data.to_pickle(path1)  

###################################################################
## split
df_train, df_tmp = train_test_split(pw_data, test_size=0.2)
df_valid, df_test = train_test_split(df_tmp, test_size=0.5)

# train, valid, test 저장(txt)
path = os.path.join(save_path, 'pw_train.pickle')
df_train.to_pickle(path)

path = os.path.join(save_path, 'pw_valid.pickle')
df_valid.to_pickle(path)

path = os.path.join(save_path, 'pw_test.pickle')
df_test.to_pickle(path)

In [ ]:
pd.read_pickle('./LearningData/type_1/pw_test.pickle')

# Type2

In [ ]:
for i in range(0,27):
    char = letter[i]
    print(char, 'Data Begin')
    
    # 데이터 로드
    load_path = './Preprocessed/' +str(char)+'.pickle'
    pw_df = pd.read_pickle(load_path)
    
    # 원하는 타입 추출
    learn_data = pw_df[type_2]
    print('length of original learn data:',len(learn_data))
    
    # 불필요한 데이터 drop 및 reindex
    learn_data = learn_data.dropna(how='any').reset_index(drop=True)
    print('length of processed learn data:',len(learn_data))
    
    # 길이가 2 이상인 데이터 나누기
    learn_data['Splitted'] = learn_data.apply(lambda x:SplitData_Multi(x[type_2[0]], x[type_2[1]]), axis = 1)
    print('Data Splitted')
    
    # 나뉘어진 데이터 이어 붙이기 
    learn_data= ListExtend(learn_data)
    
    # src, tgt 데이터 프래임 생성 및 파일 저장
    DFandSave(learn_data,2,char)
    print('')
    

In [ ]:
len(pw_data)

In [ ]:
pd.read_pickle('./LearningData/type_2/letter_specific_data/k.pickle')

In [97]:
pw_data = pd.read_pickle('./Training_Inference/LearningData/type_2/pw_all.pickle')

In [ ]:
data_columns = ['src', 'tgt']
pw_data = pd.DataFrame(columns=data_columns)

for i in range(0,27):
    char = letter[i]
    print(char, 'Data load and Append')
    
    load_path = './LearningData/type_2/letter_specific_data/'+str(char)+'.pickle'
    temp = pd.read_pickle(load_path)
    
    print(len(temp))
    pw_data = pw_data.append(temp)    

In [102]:
## type 2
save_path = type_2_path

print('%s 갯수 : %d\n' % (type_2, len(pw_data)))
print(pw_data[:5])

file_name_src = pw_data.columns[0]
file_name_tgt = pw_data.columns[1]
print('%s -> %s'%(file_name_src, file_name_tgt))

# src, tgt 전체 데이터 저장(csv)
path1 = os.path.join(save_path, 'pw_all.txt')
pw_data.to_pickle(path1)  

################################################################ split
# df_train, df_tmp = train_test_split(pw_data, test_size=0.2)
# df_valid, df_test = train_test_split(df_tmp, test_size=0.5)

# # train, valid, test 저장(txt)
# path = os.path.join(save_path, 'pw_train.txt')
# df_train.to_pickle(path)

# path = os.path.join(save_path, 'pw_valid.txt')
# df_valid.to_pickle(path)

# path = os.path.join(save_path, 'pw_test.txt')
# df_test.to_pickle(path)

['low', 'low&num'] 갯수 : 72859274

       src       tgt
0  nikolaw  nikola69
1  nikolaw  nikola11
2  nikolaw  nikola91
3  nikolaw  nikola19
4  nikolaw  nikolaw9
src -> tgt


In [101]:
save_path

'LearningData/type_2/letter_specific_data'

In [8]:
pd.read_pickle('./LearningData/type_2/pw_test.pickle')

ModuleNotFoundError: No module named 'pandas.core.internals.managers'; 'pandas.core.internals' is not a package

# Type3

In [8]:
for i in range(0,27):
    char = letter[i]
    print(char, 'Data Begin')
    
    # 데이터 로드
    load_path = './Preprocessed/' +str(char)+'.pickle'
    pw_df = pd.read_pickle(load_path)
    
    # 원하는 타입 추출
    learn_data1 = pw_df[[type_3[0], type_3[1][0]]]
    learn_data2 = pw_df[[type_3[0], type_3[1][1]]]
    print('length of original learn data:',len(learn_data1))
    
    # 불필요한 데이터 drop 및 reindex
    learn_data1 = learn_data1.dropna(how='any').reset_index(drop=True)
    learn_data2 = learn_data2.dropna(how='any').reset_index(drop=True)
    
    learn_data2.rename(columns = {'spc&low':'up&low'}, inplace=True)
    learn_data = learn_data1.append(learn_data2)
    learn_data = learn_data.reset_index(drop=True)
    
    print('length of processed learn data:',len(learn_data))
    
    # 길이가 2 이상인 데이터 나누기
    learn_data['Splitted'] = learn_data.apply(lambda x:SplitData_Multi(x[type_3[0]], x[type_3[1][0]]), axis = 1)
    print('Data Splitted')
    
    # 나뉘어진 데이터 이어 붙이기 
    learn_data= ListExtend(learn_data)
    
    # src, tgt 데이터 프래임 생성 및 파일 저장
    DFandSave(learn_data,3,char)
    print('')
    

a Data Begin
length of original learn data: 8770116
length of processed learn data: 697460
Data Splitted
0
100000
200000
300000
400000
500000
600000
a Data Saved length= 1695090

b Data Begin
length of original learn data: 4914853
length of processed learn data: 354564
Data Splitted
0
100000
200000
300000
b Data Saved length= 870858

c Data Begin
length of original learn data: 4661423
length of processed learn data: 329292
Data Splitted
0
100000
200000
300000
c Data Saved length= 864655

d Data Begin
length of original learn data: 5593731
length of processed learn data: 424525
Data Splitted
0
100000
200000
300000
400000
d Data Saved length= 1068340

e Data Begin
length of original learn data: 2715231
length of processed learn data: 219626
Data Splitted
0
100000
200000
e Data Saved length= 569694

f Data Begin
length of original learn data: 2582223
length of processed learn data: 201128
Data Splitted
0
100000
200000
f Data Saved length= 583050

g Data Begin
length of original learn data

In [10]:
pd.read_pickle('./LearningData/type_3/letter_specific_data/c.pickle')

,src,tgt
0,caberboo,Ciberbox
1,eesmpinutile,Esmpinutile
2,esmpinutilee,Esmpinutile
3,akilina,Akilina
4,gfifdjkz,Gfifdjkz
...,...,...
864650,soccerabc,soccer!
864651,mmsocceraa,soccer!
864652,czerwonyptak,czerwony.gnom
864653,czerwonygnom,czerwony.gnom


In [11]:
data_columns = ['src', 'tgt']
pw_data = pd.DataFrame(columns=data_columns)

for i in range(0,27):
    char = letter[i]
    print(char, 'Data')
    
    load_path = './LearningData/type_3/letter_specific_data/'+str(char)+'.pickle'
    temp = pd.read_pickle(load_path)
    
    print(len(temp))
    pw_data = pw_data.append(temp)    

a Data
1695090
b Data
870858
c Data
864655
d Data
1068340
e Data
569694
f Data
583050
g Data
720185
h Data
420399
i Data
432901
j Data
695772
k Data
961438
l Data
940135
m Data
1698222
n Data
745229
o Data
327205
p Data
771370
q Data
58973
r Data
715154
s Data
1741493
t Data
692045
u Data
116668
v Data
605503
w Data
226903
x Data
81432
y Data
183610
z Data
257503
num Data
244152


In [12]:
len(pw_data)

18287979

In [13]:
## type 3
save_path = type_3_path[:-20]

print('%s 갯수 : %d\n' % (type_3, len(pw_data)))
print(pw_data[:5])

file_name_src = pw_data.columns[0]
file_name_tgt = pw_data.columns[1]
print('%s -> %s'%(file_name_src, file_name_tgt))

# src, tgt 전체 데이터 저장(csv)
path1 = os.path.join(save_path, 'pw_all.pickle')
pw_data.to_pickle(path1)  

################################################################ split
df_train, df_tmp = train_test_split(pw_data, test_size=0.2)
df_valid, df_test = train_test_split(df_tmp, test_size=0.5)

# train, valid, test 저장(txt)
path = os.path.join(save_path, 'pw_train.pickle')
df_train.to_pickle(path)

path = os.path.join(save_path, 'pw_valid.pickle')
df_valid.to_pickle(path)

path = os.path.join(save_path, 'pw_test.pickle')
df_test.to_pickle(path)

['low', ['up&low', 'spc&low']] 갯수 : 18287979

                    src         tgt
0                aidana      Aidana
1                aidana      Aidana
2                aidana      Aidana
3  saifulazrisaifulazri  Saifulazri
4            saifulazri  Saifulazri
src -> tgt


# Type4

In [ ]:
'low'  ->  'up&num&low', 'spc&num&low', 'spc&up&low'

In [ ]:
type_4 = [clms[1],[clms[7],clms[11],clms[13]]]   # low -> low&up&num, low&num&spc, low&up&spc

In [96]:
for i in range(1,27):
    char = letter[i]
    print(char, 'Data Begin')
    
    # 데이터 로드
    load_path = './Preprocessed/' +str(char)+'.pickle'
    pw_df = pd.read_pickle(load_path)
    
    # 원하는 타입 추출
    learn_data1 = pw_df[[type_4[0], type_4[1][0]]]
    learn_data2 = pw_df[[type_4[0], type_4[1][1]]]
    learn_data3 = pw_df[[type_4[0], type_4[1][2]]]
    print('length of original learn data:',len(learn_data1))
    
    # 불필요한 데이터 drop 및 reindex
    learn_data1 = learn_data1.dropna(how='any').reset_index(drop=True)
    learn_data2 = learn_data2.dropna(how='any').reset_index(drop=True)
    learn_data3 = learn_data3.dropna(how='any').reset_index(drop=True)
    
    learn_data2.rename(columns = {'spc&num&low':'up&num&low'}, inplace=True)
    learn_data3.rename(columns = { 'spc&up&low':'up&num&low'}, inplace=True)
    
    learn_data = learn_data1.append(learn_data2)
    learn_data = learn_data.append(learn_data3)
    learn_data = learn_data.reset_index(drop=True)
    print('length of processed learn data:',len(learn_data))
    
    
    # 길이가 2 이상인 데이터 나누기
    learn_data['Splitted'] = learn_data.apply(lambda x:SplitData_Multi(x[type_4[0]], x[type_4[1][0]]), axis = 1)
    print('Data Splitted')
    
    # 나뉘어진 데이터 이어 붙이기 
    learn_data= ListExtend(learn_data)
    
    # src, tgt 데이터 프래임 생성 및 파일 저장
    DFandSave(learn_data,4,char)
    print('')
    

b Data Begin
length of original learn data: 4914853
length of processed learn data: 266475
Data Splitted
0
100000
200000
b Data Saved length= 490938

c Data Begin
length of original learn data: 4661423
length of processed learn data: 244713
Data Splitted
0
100000
200000
c Data Saved length= 451386

d Data Begin
length of original learn data: 5593731
length of processed learn data: 336696
Data Splitted
0
100000
200000
300000
d Data Saved length= 647038

e Data Begin
length of original learn data: 2715231
length of processed learn data: 144759
Data Splitted
0
100000
e Data Saved length= 275029

f Data Begin
length of original learn data: 2582223
length of processed learn data: 134653
Data Splitted
0
100000
f Data Saved length= 247656

g Data Begin
length of original learn data: 3284478
length of processed learn data: 194932
Data Splitted
0
100000
g Data Saved length= 364102

h Data Begin
length of original learn data: 2382683
length of processed learn data: 113786
Data Splitted
0
100000


In [84]:
pd.read_pickle('./LearningData/type_4/letter_specific_data/g.pickle')

,src,tgt
0,asdzxc,Asdzxc1
1,asdzxc,asdzxc1Q
2,vmogile,Vmogile2011
3,kimgorgv,22KIMgorGV35
4,whatever,Whatever4
...,...,...
364097,gysoftq,Gy_soft
364098,gysoftq,Gy_softa
364099,gyamfua,Gyamfuah@yahoo.com
364100,gyamfuaa,Gyamfuah@yahoo.com


In [85]:
data_columns = ['src', 'tgt']
pw_data = pd.DataFrame(columns=data_columns)

for i in range(0,27):
    char = letter[i]
    print(char, 'Data')
    
    load_path = './LearningData/type_4/letter_specific_data/'+str(char)+'.pickle'
    temp = pd.read_pickle(load_path)
    
    print(len(temp))
    pw_data = pw_data.append(temp)    

a Data
1004286
b Data
490938
c Data
451386
d Data
647038
e Data
275029
f Data
247656
g Data
364102
h Data
193338
i Data
196451
j Data
364371
k Data
534378
l Data
401945
m Data
848450
n Data
408557
o Data
145351
p Data
371446
q Data
42571
r Data
383178
s Data
937347
t Data
354075
u Data
48070
v Data
307436
w Data
117017
x Data
39084
y Data
87186
z Data
139404
num Data
64203


In [86]:
len(pw_data)

9464293

In [88]:
## type 4
save_path = type_4_path[:-20]

print('%s 갯수 : %d\n' % (type_4, len(pw_data)))
print(pw_data[:5])

file_name_src = pw_data.columns[0]
file_name_tgt = pw_data.columns[1]
print('%s -> %s'%(file_name_src, file_name_tgt))

# src, tgt 전체 데이터 저장(csv)
path1 = os.path.join(save_path, 'pw_all.pickle')
pw_data.to_pickle(path1)  

################################################################ split
df_train, df_tmp = train_test_split(pw_data, test_size=0.2)
df_valid, df_test = train_test_split(df_tmp, test_size=0.5)

# train, valid, test 저장(txt)
path = os.path.join(save_path, 'pw_train.pickle')
df_train.to_pickle(path)

path = os.path.join(save_path, 'pw_valid.pickle')
df_valid.to_pickle(path)

path = os.path.join(save_path, 'pw_test.pickle')
df_test.to_pickle(path)

['low', ['up&num&low', 'spc&num&low', 'spc&up&low']] 갯수 : 9464293

        src           tgt
0  aidardia  aiDAR7aidar5
1  aidardia  aiDAR7aidar5
2  aidardia  aiDAR7aidar5
3  barabash  Barabash2011
4   android     Android06
src -> tgt


# Type5

In [ ]:
pw_df['num']

In [ ]:
for i in range(0,27):
    char = letter[i]
    print(char, 'Data Begin')
    
    # 데이터 로드
    load_path = './Preprocessed/' +str(char)+'.pickle'
    pw_df = pd.read_pickle(load_path)
    
    # 원하는 타입 추출
    learn_data = pw_df[type_5]
    print('length of original learn data:',len(learn_data))
    
    # 불필요한 데이터 drop 및 reindex
    learn_data = DropReindex(learn_data, type_5)
    print('length of processed learn data:',len(learn_data))
    
    # 길이가 2 이상인 데이터 나누기
    learn_data['Splitted'] = learn_data['num'].apply(SplitData)
    print('Data Splitted')
    
    # 나뉘어진 데이터 이어 붙이기 
    learn_data= ListExtend(learn_data)
    
    # src, tgt 데이터 프래임 생성 및 파일 저장
    DFandSave(learn_data,5,char)
    print('')
    

In [ ]:
# 올바른 전처리 결과 확인용
pd.read_pickle('./LearningData/type_5/letter_specific_data/k.pickle')

In [ ]:
# 전처리된 데이터 a~z, num 들을 로드한 후 1개의 df로 결합

data_columns = ['src', 'tgt']
pw_data = pd.DataFrame(columns=data_columns)

for i in range(0,27):
    char = letter[i]
    print(char, 'Data load and Append')
    
    load_path = './LearningData/type_5/letter_specific_data/'+str(char)+'.pickle'
    temp = pd.read_pickle(load_path)
    
    print(len(temp))
    pw_data = pw_data.append(temp)    

In [ ]:
## type 5
save_path = type_5_path

print('%s 갯수 : %d\n' % (type_5, len(pw_data)))
print(pw_data[:5])

file_name_src = pw_data.columns[0]
file_name_tgt = pw_data.columns[1]
print('%s -> %s'%(file_name_src, file_name_tgt))

# src, tgt 모두 저장(csv)
path1 = os.path.join(save_path, 'pw_all.pickle')
pw_data.to_pickle(path1)  

########################################################## split
df_train, df_tmp = train_test_split(pw_data, test_size=0.2)
df_valid, df_test = train_test_split(df_tmp, test_size=0.5)

# train, valid, test 저장(txt)
path = os.path.join(save_path, 'pw_train.pickle')
df_train.to_pickle(path)

path = os.path.join(save_path, 'pw_valid.pickle')
df_valid.to_pickle(path)

path = os.path.join(save_path, 'pw_test.pickle')
df_test.to_pickle(path)

# Type6

In [64]:
for i in range(0,27):
    char = letter[i]
    print(char, 'Data Begin')
    
    # 데이터 로드
    load_path = './Preprocessed/' +str(char)+'.pickle'
    pw_df = pd.read_pickle(load_path)
    
    # 원하는 타입 추출
    learn_data = pw_df[type_6]
    print('length of original learn data:',len(learn_data))
    
    # 불필요한 데이터 drop 및 reindex
    learn_data = learn_data.dropna(how='any').reset_index(drop=True)
    print('length of processed learn data:',len(learn_data))
    
    # 길이가 2 이상인 데이터 나누기
    learn_data['Splitted'] = learn_data.apply(lambda x:SplitData_Multi(x[type_6[0]], x[type_6[1]]), axis = 1)
    print('Data Splitted')
    
    # 나뉘어진 데이터 이어 붙이기 
    learn_data= ListExtend(learn_data)
    
    # src, tgt 데이터 프래임 생성 및 파일 저장
    DFandSave(learn_data,6,char)
    print('')
    

a Data Begin
length of original learn data: 8770116
length of processed learn data: 816899
Data Splitted
0
100000
200000
300000
400000
500000
600000
700000
800000
a Data Saved length= 3685846

b Data Begin
length of original learn data: 4914853
length of processed learn data: 318057
Data Splitted
0
100000
200000
300000
b Data Saved length= 1138368

c Data Begin
length of original learn data: 4661423
length of processed learn data: 258065
Data Splitted
0
100000
200000
c Data Saved length= 908324

d Data Begin
length of original learn data: 5593731
length of processed learn data: 419694
Data Splitted
0
100000
200000
300000
400000
d Data Saved length= 1461795

e Data Begin
length of original learn data: 2715231
length of processed learn data: 211606
Data Splitted
0
100000
200000
e Data Saved length= 748726

f Data Begin
length of original learn data: 2582223
length of processed learn data: 203776
Data Splitted
0
100000
200000
f Data Saved length= 746754

g Data Begin
length of original le

In [92]:
pd.read_pickle('./LearningData/type_6/letter_specific_data/k.pickle')

,src,tgt
0,19800222,198005kdw
1,123456,123456qazwsx
2,123456,1123456qazwsx
3,123456,123456qazwsxx
4,123456,123456qazwsx1
...,...,...
1492575,617383,617383bogdan
1492576,02071988,alex02071988
1492577,02071988,alex12121688
1492578,02071988,alex020719


In [93]:
data_columns = ['src', 'tgt']
pw_data = pd.DataFrame(columns=data_columns)

for i in range(0,27):
    char = letter[i]
    print(char, 'Data')
    
    load_path = './LearningData/type_6/letter_specific_data/'+str(char)+'.pickle'
    temp = pd.read_pickle(load_path)
    
    print(len(temp))
    pw_data = pw_data.append(temp)    

a Data
3685846
b Data
1138368
c Data
908324
d Data
1461795
e Data
748726
f Data
746754
g Data
950309
h Data
590339
i Data
655794
j Data
737214
k Data
1492580
l Data
1226515
m Data
2081044
n Data
1082010
o Data
511483
p Data
894682
q Data
136127
r Data
891575
s Data
2296091
t Data
908942
u Data
197513
v Data
933019
w Data
440987
x Data
183499
y Data
362702
z Data
485979
num Data
1836728


In [94]:
len(pw_data)

27584945

In [95]:
## type 6
save_path = type_6_path[:-20]

print('%s 갯수 : %d\n' % (type_6, len(pw_data)))
print(pw_data[:5])

file_name_src = pw_data.columns[0]
file_name_tgt = pw_data.columns[1]
print('%s -> %s'%(file_name_src, file_name_tgt))

# src, tgt 전체 데이터 저장(csv)
path1 = os.path.join(save_path, 'pw_all.pickle')
pw_data.to_pickle(path1)  

################################################################ split
df_train, df_tmp = train_test_split(pw_data, test_size=0.2)
df_valid, df_test = train_test_split(df_tmp, test_size=0.5)

# train, valid, test 저장(txt)
path = os.path.join(save_path, 'pw_train.pickle')
df_train.to_pickle(path)

path = os.path.join(save_path, 'pw_valid.pickle')
df_valid.to_pickle(path)

path = os.path.join(save_path, 'pw_test.pickle')
df_test.to_pickle(path)

['num', 'low&num'] 갯수 : 27584945

           src               tgt
0   1234567890        15678902q1
1    123456789       ai123456789
2    123456789  aidos12345678901
3    123456789        a123456789
4  12345678910       ai123456789
src -> tgt
